# Library imports

In [15]:
import pandas as pd
import numpy as np
import inflection
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import warnings
import xgboost as xgb
import random
import pickle
import requests
import json

from sklearn.preprocessing import MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.ensemble      import RandomForestRegressor
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.metrics       import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from pycorrcat.pycorrcat   import plot_corr, corr_matrix
from tabulate              import tabulate
from IPython.display       import Image
from boruta                import BorutaPy


# Helper functions

In [ ]:
def jupyter_settings():
    
    pd.options.display.max_columns = None   
    


warnings.filterwarnings("ignore")
jupyter_settings()



def ml_error(model_name, y_test, y_predicted):
    mae = mean_absolute_error(y_test, y_predicted)
    mape = mean_absolute_percentage_error(y_test, y_predicted)
    rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
    
    return pd.DataFrame({'Model name': model_name,
                         'MAE':mae,
                         'MAPE':mape,
                         'RMSE': rmse}, index=[0])
    


def mean_percentage_error(y_test, y_predicted):
    return np.mean((y_test- y_predicted)/y_test)
    
    
def cross_validation_Time_Series(k, X_train_CrossVal, y_train, instantiated_model, model_name, verbose=False):

  mae_list = []
  mape_list = []
  rmse_list = []  


  for k in reversed(range(1,(k+1))):  


    # Filtros de datas para dividir os sets de treino e validação

    validation_set_start_date = X_train_CrossVal['date'].max() - datetime.timedelta(days=k*6*7)

    validation_set_end_date = X_train_CrossVal['date'].max() - datetime.timedelta(days=(k-1)*6*7)


    # X de treino e validação

    X_validation_CV = X_train_CrossVal[(X_train_CrossVal['date']>=validation_set_start_date) & (X_train_CrossVal['date']<validation_set_end_date)]

    X_train_CV = X_train_CrossVal[X_train_CrossVal['date'] < validation_set_start_date]

    X_validation_CV = X_validation_CV.drop(columns=['date']) #remover a coluna 'date'
    X_train_CV = X_train_CV.drop(columns=['date']) #remover a coluna 'date'



    # y de treino e validação (usei os índices das linhas do X_validation_CV e X_train_CV para filtrar o y)

    y_validation_CV = y_train[X_validation_CV.index]

    y_train_CV = y_train[X_train_CV.index]

    
    # Model training

    model = instantiated_model.fit(X=X_train_CV, y=y_train_CV )

    y_predicted_CV = model.predict(X_validation_CV)

    model_error = ml_error(model_name, np.expm1(y_validation_CV), np.expm1(y_predicted_CV ))

    if verbose:
      print(model_error)

    mae_list.append(model_error['MAE'])
    mape_list.append(model_error['MAPE'])
    rmse_list.append(model_error['RMSE'])

    

  return pd.DataFrame({'Model name': model_name,
                            'MAE CV': str(np.round(np.mean(mae_list), 2)) + ' +/- ' + str(np.round(np.std(mae_list, ddof=(k-1)), 2)),
                            'MAPE CV': str(np.round(np.mean(mape_list), 2)) + ' +/- ' + str(np.round(np.std(mape_list, ddof=(k-1)), 2)),
                            'RMSE CV': str(np.round(np.mean(rmse_list), 2)) + ' +/- ' + str(np.round(np.std(rmse_list, ddof=(k-1)), 2))},
                            index=[0])
    

# Data import and basic inspection

##  <font color="#808080">data loading and checking</font>

In [ ]:
df_train = pd.read_csv('/home/gustavo/repos/Rossmann/train.csv', low_memory=False)

df_train.head()

In [ ]:
# Checking data dimensions

print('\nData dimensions:\n')
print('Number of rows:{}'.format(df_train.shape[0]))
print('Number of columns:{}'.format(df_train.shape[1]))

In [ ]:
df_store = pd.read_csv('/home/gustavo/repos/Rossmann/store.csv', low_memory=False)

df_store.head()

In [ ]:
# Checking data dimensions

print('\nData dimensions:\n')
print('Number of rows:{}'.format(df_store.shape[0]))
print('Number of columns:{}'.format(df_store.shape[1]))

In [ ]:
# Merging df's

df = pd.merge(df_train, df_store, how='left', on='Store')

df.head()

In [ ]:
# Checking data dimensions after merging

print('\nData dimensions:\n')
print('Number of rows:{}'.format(df.shape[0]))
print('Number of columns:{}'.format(df.shape[1]))

In [ ]:
# Checking data types

df.dtypes

In [ ]:
# Changing column 'Date' to datetime

df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Adjusting column names

df.columns = list(map(lambda x: inflection.underscore(x), df.columns)) #changing to underscore + lower(snakecase)

## <font color="#808080">filling out NA's</font>

In [ ]:
# Checking % of missing values

df.isna().sum() / df.shape[0]

In [ ]:
# Filling NA's on 'competition_distance' column ===========================================


## LÓGICA USADA: se a distância está NA assumiu-se que a distância da loja concorrente é muita alta.
### então foi feito o preenchimento por um valor acima da distância mais alta do dataset

df['competition_distance'] = df['competition_distance'].apply(lambda x: 200000.0 if pd.isna(x) else x)



# Filling NA's on 'competition_open_since_month' column ==================================


## LÓGICA USADA: completou-se o valor NA usando o mês da coluna 'date' (data da venda) da respectiva linha

df['competition_open_since_month'] = df[['date','competition_open_since_month']].apply(lambda x: x['date'].month if pd.isna(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)



# Filling NA's on 'competition_open_since_year' column =====================================


## LÓGICA USADA: completou-se o valor NA usando o ano da coluna 'date' (data da venda) da respectiva linha

df['competition_open_since_year'] = df[['date','competition_open_since_year']].apply(lambda x: x['date'].year if pd.isna(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)



# Filling NA's on 'promo2_since_week' column =================================================

## LÓGICA USADA: todos os valores onde a coluna promo2 estavam 0 (não houve promoção na loja) a coluna
### 'promo2_since_week' estava NA, então assumiu-se que não houve promoção nessa loja e preencheu-se com 0

df['promo2_since_week'] = df['promo2_since_week'].fillna(0)



# Filling NA's on 'promo2_since_year' column ===================================================

## LÓGICA USADA: todos os valores onde a coluna promo2 estavam 0 (não houve promoção na loja) a coluna
### 'promo2_since_year' estava NA, então assumiu-se que não houve promoção nessa loja e preencheu-se com 0

df['promo2_since_year'] = df['promo2_since_year'].fillna(0)




# Filling NA's on 'promo_interval' column ========================================================

## LÓGICA USADA: todos os valores onde a coluna promo2 estavam 0 (não houve promoção na loja) a coluna
### 'promo_interval' estava NA, então assumiu-se que não houve promoção nessa loja e preencheu-se com 0

df['promo_interval'] = df['promo_interval'].fillna(0)



# Abaixo será criado um dicionário que auxiliará na conversão dos meses de números em letras:

# criando mapeamento dos meses
month_map = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}


# criado a coluna 'month_map'
## LÒGICA: extraíu-se o mês (em número) da coluna 'date' e converteu-se em mês (em palavra) colocando em uma nova
### coluna

df['month_map'] = df['date'].dt.month.map(month_map)


# criando a coluna 'is_promo'
# LÓGICA: Se o dia da venda (coluna 'data') estiver em um mês onde foi feita a promoção (coluna 'promo_interval')
## então será atribuído 1 na nova coluna 'is_promo', se não tiver promoção ou a venda tiver sido feita em um mês
## fora da promoção então o valor atribuído será 0

df['is_promo'] = df[['month_map', 'promo_interval']].apply(lambda x: 0 if x['promo_interval']==0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)


In [ ]:
# There is no more NA's in the dataset

df.isna().sum() / df.shape[0]

In [ ]:
# Changing data types after previous columns creation

## columns 'competition_open_since_month','competition_open_since_year','promo2_since_week','promo2_since_year'
## need to be changed to int64

df.dtypes

In [ ]:
# Changing data types after previous columns creation

df[['competition_open_since_month',
    'competition_open_since_year',
    'promo2_since_week',
    'promo2_since_year']] = df[['competition_open_since_month',
                                'competition_open_since_year',
                                'promo2_since_week',
                                'promo2_since_year']].astype(int)

## <font color="#808080">descriptive statistics</font>

In [ ]:
# Separando df com atributos númericos e categóricos

num_attrib = df.select_dtypes(include=['float64','int64'])

cat_attrib = df.select_dtypes(exclude=['float64','int64','datetime64[ns]'])

In [ ]:
# Descriptive statistics

pd.DataFrame(data=[num_attrib.apply(np.min),
          num_attrib.apply(np.max),
          num_attrib.apply(lambda x: np.max(x) - np.min(x)),
          num_attrib.apply(np.mean),
          num_attrib.apply(np.median),
          num_attrib.apply(np.std),
          num_attrib.apply(lambda x: x.skew()),
          num_attrib.apply(lambda x: x.kurtosis())],
             
             index=np.array(['min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']))

In [ ]:
# Numeric atribute view:

# Exploring 'competition_distance'

sns.histplot(df.loc[df['competition_distance']<75000, 'competition_distance']);

In [ ]:
# Numeric atribute view:

# Exploring 'sales'

sns.histplot(df['sales'])

In [ ]:
# Numeric atribute view:

# Exploring 'customers'

sns.histplot(df['customers'])

In [ ]:
# Categorical atribute view:

plt.figure(figsize=(15,10))

plt.subplot(2,3,1)
sns.boxplot(x=df['store_type'], y=df['sales'], data=df)

plt.subplot(2,3,2)
sns.boxplot(x=df.loc[(df['state_holiday']!='0') & (df['sales']>0), 'state_holiday'], y=df['sales'], data=df)

plt.subplot(2,3,3)
sns.boxplot(x=df['assortment'], y=df['sales'], data=df)



plt.subplot(2,3,4)
sns.boxplot(x=df['store_type'], y=df['customers'], data=df)

plt.subplot(2,3,5)
sns.boxplot(x=df.loc[(df['state_holiday']!='0') & (df['customers']>0), 'state_holiday'], y=df['customers'], data=df)

plt.subplot(2,3,6)
sns.boxplot(x=df['assortment'], y=df['customers'], data=df)



plt.tight_layout()


# Feature engineering

## <font color="#808080">mindmap de hipóteses</font>

In [ ]:
Image(filename='/home/gustavo/repos/Rossmann/Dailly_store_sales_mindmap.png')

## <font color="#808080">hipóteses à serem validadas com os dados presentes no dataset</font>

 <br></br>
 Nessa parte o mapa mental de hipóteses acima deve ser observado. 
 <br></br>

A partir dos atributos de cada entidade devem ser levantadas as hipóteses que impactam no fenômeno a ser modelado.

LÓGICA: Um aumento/diminuição no atributo x aumenta/diminui a quantidade de vendas.
<br></br>

As hipóteses levantadas são posteriormente validadas a partir dos dados disponíveis no dataset


Dessa forma, baseado nos dados disponíveis no dataset foram levantadas as seguintes hipóteses:
<br></br>

Entidade <em>Loja</em>:


**1.** Lojas com maior sortimentos deveriam vender mais.

**2.** Lojas com competidores mais próximos deveriam vender menos.

**3.** Lojas com competidores à mais tempo deveriam vendem mais.
<br></br>

Entidade <em>Produtos</em>:

**4.** Lojas com promoções ativas por mais tempo deveriam vender mais.

**5.** Lojas com mais dias de promoção deveriam vender mais.

**6.** Lojas com mais promoções consecutivas deveriam vender mais.
<br></br>

Entidade <em>Temporal</em>:

**7.** Lojas abertas durante o feriado de Natal deveriam vender mais.

**8.** Lojas deveriam vender mais ao longo dos anos.

**9.** Lojas deveriam vender mais no segundo semestre do ano.

**10.** Lojas deveriam vender mais depois do dia 10 de cada mês.

**11.** Lojas deveriam vender menos aos finais de semana.

**12.** Lojas deveriam vender menos durante os feriados escolares.

In [ ]:
df_1 = df.copy()

In [ ]:
# Changing column 'assortment'

df_1['assortment'] = df_1['assortment'].apply(lambda x:'basic' if x=='a' else 'extra' if x=='b' else 'extended')


# Changing column 'state_holiday'

df_1['state_holiday'] = df_1['state_holiday'].apply(lambda x:'Public holiday' if x=='a' else 'Easter' if x=='b' else 'Christmas' if x=='c' else 'regular day')


In [ ]:
# Derivando novas variáveis:


# year

df_1['year'] = df_1['date'].dt.year


# month

df_1['month'] = df_1['date'].dt.month


# day

df_1['day'] = df_1['date'].dt.day


# week of year

df_1['week_of_year'] = df_1['date'].dt.weekofyear


# year week
 
df_1['year_week'] = df_1['date'].dt.strftime('%Y-%W') # esse comando apenas coleta a data da venda e muda a formatação




# competition_since (quanto tempo desde o início da competição ate a data da compra)

# 1o passo - juntar as colunas 'competition_open_since_month' com 'competition_open_since_year'

df_1['competition_since'] = df_1[['competition_open_since_year','competition_open_since_month']].apply(lambda x: datetime.datetime(year= x['competition_open_since_year'], month=x['competition_open_since_month'], day=1), axis=1)

# 2o passo - subtrair as datas de 'date' e 'competition_since' e dividir 30 (granularidade mês)
df_1['competition_time_month'] = ((df_1['date'] - df_1['competition_since'])/ 30).apply(lambda x: x.days).astype(int)




# promo2_since (quanto tempo desde a a promoção estar ativa até a data da compra)

# 1° passo: juntar a coluna 'promo2_since_year' com a coluna 'promo2_since_week' e armazenar em uma nova coluna
df_1['promo2_since'] = df_1.apply(lambda x: str(x['promo2_since_year']) + '-' + str(x['promo2_since_week']),axis=1)

# 2° passo: converter 'promo_since' de string para data
# OBSERVAÇÃO IMPORTANTE: tive que fazer uma adaptação no código.
# no código original a atribuição é feita sobre todas as linhas da coluna 'promo_since', no entanto o lambda
# não estava funcionando na coluna 'promo_since' nas linhas com valores zerados '0-0' (proveniente dos valores
# NA's que foram substituídos por 0 na etapa de preenchimento de NA's) então tive que aplicar a função no df
# filtrado (df_1['promo_since']!='0-0')
df_1.loc[df_1['promo2_since']!='0-0','promo2_since'] = df_1.loc[df_1['promo2_since']!='0-0','promo2_since'].apply(lambda x: datetime.datetime.strptime(x+'-1', '%Y-%W-%w') - datetime.timedelta(days=7) )

# 3° passo: subtrair a data da coluna 'date' da coluna 'promo_since' (criada no passo acima), mantendo o filtro
# condicional de linhas (df_1['promo_since']!='0-0') a atribuindo a uma nova coluna 'promo_time_week' mantendo
# o filtro de linhas
df_1.loc[df_1['promo2_since']!='0-0','promo2_time_week'] = ((df_1.loc[df_1['promo2_since']!='0-0','date'] - df_1.loc[df_1['promo2_since']!='0-0','promo2_since']) / 7).apply(lambda x: x.days).astype(int)

# 4° passo: os filtros de linha anteriores resultaram apenas em dados preenchidos nas linhas do filtro. As linhas
# não pertencentes ao filtro foram preenchidas com NA's. Preencher esses valores com 0
df_1['promo2_time_week'] = df_1['promo2_time_week'].fillna(0)



# Variable filtering and variable selection

<br></br>
<center>Variable filtering x Variable selection</center>
<br></br>

<div align="justify"><b>Variable filtering</b> - eliminação de variáveis baseada nas restrições do negócio.

ex: variáveis que estão disponíveis no dataset de treino mas não estarão em produção, no momento da previsão.
<br></br>

<b>Variable selection</b> - eliminação de variáveis irrelevantes para o aprendizado.

ex: nesse caso serão eliminadas as colunas auxiliares que foram criadas durante a manipulação dos dados</div>
<br></br>
<br></br>

<center>Eliminação de linhas</center>
<br></br>


<div align="justify">Também é importante eliminar as linhas irrelevantes no dataset.
ex: para esse caso serão eliminadas as linhas onde o valor na coluna 'open' é 0 (loja fechada).
As informações de venda nos dias em que a loja está fechada são irrelevantes.
Após essa etapa a coluna 'open' será também eliminada pois ela terá valor constante, sendo irrelevante para o aprendizado de máquina.</div>



## <font color="#808080">Seleção das linhas</font>

In [ ]:
df_2 = df_1.copy()

In [ ]:
# Quando a loja está fechada ('open'==0) a quantidade de vendas e clientes é 0. Eliminar essas linhas.
# eliminar também as linhas onde não houve vendas (df_2['sales']==0)

df_2 = df_2[(df_2['open']!=0) & (df_2['sales']>0)]

## <font color="#808080">Seleção das colunas</font>

In [ ]:
# Eliminar a coluna 'customers' (no momento do modelo em produção a informação sobre a quantidade
# de clientes na loja não estará disponível).
# coluna 'open' estará com valor constante (1 - loja aberta)
# demais colunas são auxiliares

df_2 = df_2.drop(['customers','open','promo_interval','month_map'], axis=1)


# EDA

## <font color="#808080">Análise univariada</font>

### <font color="#808080">Numerical attributes</font>

In [ ]:
# Numerical variables

df_2.select_dtypes(include=['float64','int64']).head()

 <br></br>
 <font size="+2">Histogram: Response variable</font>
 <br></br>

In [ ]:
# Response variable distribution

plt.figure(figsize=(10,5))
sns.distplot(df_2['sales']);

<br></br>
<font size="+2">Histogram: numeric attributes</font>
<br></br>

In [ ]:
# Numerical attributes distribution

df_2.select_dtypes(include=['float64','int64']).hist(figsize=(15,10), color='grey', bins=25)
plt.tight_layout();

* 'store', 'promo_2' são praticamente constantes não apresentando valores/informação relevante

* 'competition_distance' a maioria dos valores são próximos de zero, indicando que as lojas jão bastante próximas de seus competidores.

* 'day' observa-se picos em intervalos regulares. Possívelmente finais de semana

### <font color="#808080">Categorical attributes</font>

In [ ]:
# Selecionando atributos categóricos

df_2.select_dtypes(exclude=['float64','int64','datetime64[ns]']).head()

 <br></br>
 <font size="+2">Countplot and Kde plots:</font>
 <br></br>

In [ ]:
# Countplot


# 'state_holiday'

# nesse plot removi as vendas da categoria 'regular day' pq as vendas eram bem maiores que os demais dias e
## não estava dando para visualizar as vendas dos feriados

plt.figure(figsize=(15,8))

plt.subplot(2,3,1)
sns.countplot(df_2.select_dtypes(exclude=['float64','int64','datetime64[ns]'])['state_holiday'][df_2.select_dtypes(exclude=['float64','int64','datetime64[ns]'])['state_holiday']!='regular day'],
             order = df_2.select_dtypes(exclude=['float64','int64','datetime64[ns]'])['state_holiday'][df_2.select_dtypes(exclude=['float64','int64','datetime64[ns]'])['state_holiday']!='regular day'].value_counts().index[::-1])


# assortment

plt.subplot(2,3,2)
sns.countplot(df_2.select_dtypes(exclude=['float64','int64','datetime64[ns]'])['assortment'],
             order = df_2.select_dtypes(exclude=['float64','int64','datetime64[ns]'])['assortment'].value_counts().index[::-1])


# store_type

plt.subplot(2,3,3)
sns.countplot(df_2.select_dtypes(exclude=['float64','int64','datetime64[ns]'])['store_type'],
             order = df_2.select_dtypes(exclude=['float64','int64','datetime64[ns]'])['store_type'].value_counts().index[::-1])



# Kde plot

# Receita de vendas ('sales') em cada 'state_holiday'

plt.subplot(2,3,4)
sns.kdeplot(df_2[df_2['state_holiday']=='Public holiday']['sales'], 
            label='Public holiday', 
            data = df_2,
            shade=True)

sns.kdeplot(df_2[df_2['state_holiday']=='Easter']['sales'],
            label='Easter',
            data = df_2,
            shade=True)

sns.kdeplot(df_2[df_2['state_holiday']=='Christmas']['sales'],
            label='Christmas',
            data = df_2,
            shade=True)

plt.legend().set_title('state_holiday')


# Receita de vendas ('sales') em cada 'assortment'

plt.subplot(2,3,5)
sns.kdeplot(df_2[df_2['assortment']=='extra']['sales'], 
            label='extra', 
            data = df_2,
            shade=True)

sns.kdeplot(df_2[df_2['assortment']=='extended']['sales'],
            label='extended',
            data = df_2,
            shade=True)

sns.kdeplot(df_2[df_2['assortment']=='basic']['sales'],
            label='basic',
            data = df_2,
            shade=True)

plt.legend().set_title('assortment')


# Receita de vendas ('sales') em cada 'store_type'

plt.subplot(2,3,6)
sns.kdeplot(df_2[df_2['store_type']=='a']['sales'], 
            label='a', 
            data = df_2,
            shade=True)

sns.kdeplot(df_2[df_2['store_type']=='b']['sales'],
            label='b',
            data = df_2,
            shade=True)

sns.kdeplot(df_2[df_2['store_type']=='c']['sales'],
            label='c',
            data = df_2,
            shade=True)

sns.kdeplot(df_2[df_2['store_type']=='d']['sales'],
            label='d',
            data = df_2,
            shade=True)

plt.legend().set_title('store_type')

plt.tight_layout()

## <font color="#808080">Análise bivariada</font>

<br></br>
 <font size="+2">Validação das hipóteses levantadas :</font>
 <br></br>

<font size="+2">- Entidade <i>Loja</i> :</font>

<font size="+1">Hipótese 1:</font>

<b><i>Lojas com maior sortimentos deveriam vender mais.</i></b>

Análise abaixo mostra que a loja com maior sortimento (extra) é a que vende menos enquanto que a loja de sortimento básico é a que vende mais.

Portanto essa afirmação é <font color="red"><b>falsa</b></font>.

Relevância: Relevante.



In [ ]:
#**1.** Lojas com maior sortimentos deveriam vender mais.


# Barplot soma de vendas

aux1 = df_2[['assortment','sales']].groupby('assortment').sum().reset_index()

plt.figure(figsize=(15,8))


sns.barplot(x=aux1['assortment'],y=aux1['sales'],data=aux1,
           order= df_2[['assortment','sales']].groupby('assortment').sum().sort_values(by='sales').index);



<b><i>As vendas baixas do sortimento 'extra' estariam associados a algum período em específico?</i></b>

O gráfico de linhas abaixo mostra vendas para o sortimento 'extra' abaixo dos demais durante todo o período amostrado.

Uma expansão na região do sortimento 'extra' mostra que embora as vendas sejam menores elas seguem a mesma tendência dos demais sortimentos.

In [ ]:
# Será que as vendas baixas do assortimento estão associados a alguma época em específico?

# df_2[['sales','assortment','year_week']].groupby(['year_week','assortment']).sum().reset_index()


# Plot soma de vendas ao longo das semanas do ano por tipo de sortimento

fig, axes = plt.subplots(2,1, figsize=(15,10))

df_2[['sales',
      'assortment',
      'year_week']].groupby(['year_week',
                             'assortment']).sum().reset_index().pivot(index='year_week', 
                                                                      columns='assortment',
                                                                      values='sales').plot(
                                                                                          ax=axes[0])


# Plot soma de vendas ao longo das semanas do ano apenas para tipo de sortimento 'extra'


df_2[['sales',
      'assortment',
      'year_week']].groupby(['year_week',
                             'assortment']).sum().reset_index().pivot(index='year_week', 
                                                                      columns='assortment',
                                                                      values='sales')['extra'].plot(color='green',
                                                                                                   ax=axes[1]);


<font size="+1">Hipótese 2:</font>

<b><i>Lojas com competidores mais próximos deveriam vender menos.</i></b>

Análise abaixo (esquerda) mostra que as maiores quantidades de vendas estão associadas a lojas com competidores mais próximos e a medida em que a distância para o competidor mais próximo aumenta as vendas diminuem. A figura a direta do painel sugere uma correlação negativa entre distância e vendas.

Portanto essa afirmação é <font color="red"><b>falsa</b></font>.

Relevância: Media.



In [ ]:
#**2.** Lojas com competidores mais próximos deveriam vender menos.


plt.figure(figsize=(15,6))


# Scatterplot distância competidor X vendas

plt.subplot(1,2,1)
sns.scatterplot(x=df_2['competition_distance'],y=df_2['sales'], color='grey');

# Heatmap de correlação

plt.subplot(1,2,2)
sns.heatmap(df_2[['competition_distance','sales']].corr(), annot=True);

<font size="+1">Hipótese 3:</font>

<b><i>Lojas com competidores à mais tempo deveriam vender mais.</i></b>

A figura abaixo mostra que as competições mais recentes em tempo estão associadas a quantidade de vendas maiores e a medida que o tempo avança a quantidade de vendas diminui.

Portanto essa afirmação é <font color="red"><b>falsa</b></font>.

Relevância: media.



In [ ]:
# Vendas em função do tempo de concorrência

df_2[['competition_time_month','sales']].groupby('competition_time_month').sum().reset_index()[(df_2[['competition_time_month','sales']].groupby('competition_time_month').sum().reset_index()['competition_time_month']<50) & (df_2[['competition_time_month','sales']].groupby('competition_time_month').sum().reset_index()['competition_time_month']>-50)].set_index('competition_time_month').plot(figsize=(15,5));

Correlação entre as vendas com o tempo de abertura de uma loja concorrente

In [ ]:
# Correlação entre as vendas com o tempo de abertura de uma loja concorrente

sns.heatmap(df_2[['competition_time_month','sales']].groupby('competition_time_month').sum().reset_index()[df_2[['competition_time_month','sales']].groupby('competition_time_month').sum().reset_index()['competition_time_month']<0].corr(),
           annot=True);

Correlação entre as vendas com o tempo de concorrencia com um concorrente já existente no mercado

In [ ]:
# Correlação entre as vendas com o tempo de concorrencia com um concorrente já existente no mercado

sns.heatmap(df_2[['competition_time_month','sales']].groupby('competition_time_month').sum().reset_index()[df_2[['competition_time_month','sales']].groupby('competition_time_month').sum().reset_index()['competition_time_month']>0].corr(),
           annot=True);

<font size="+2">- Entidade <i>Produtos</i> :</font>

<font size="+1">Hipótese 4:</font>

<b><i>Lojas com promoções ativas por mais tempo deveriam vender mais.</i></b>

Análise abaixo mostra que as vendas são maiores quanto mais próximos do período da promoção extendida elas se encontram.
no gráfico a esquerda (antes da promoção) nota-se que a medida que a data do início da promoção se aproxima a quantidade de vendas aumenta enquanto que no gráfico da direita mostra uma queda de vendas após um certo tempo do início da promoção.

Portanto essa afirmação é <font color="red"><b>falsa</b></font>.

Relevância: Não relevante.



In [ ]:
fig, axis = plt.subplots(1,2)


# Antes de a promoção começar

df_2[['promo2_time_week','sales']].groupby('promo2_time_week').sum().reset_index()[df_2[['promo2_time_week','sales']].groupby('promo2_time_week').sum().reset_index()['promo2_time_week']<0].set_index('promo2_time_week').plot(kind='bar',
figsize=(15,5), color='grey', rot=90, ax=axis[0], title='before promo');

# Depois que a promoção começou

df_2[['promo2_time_week','sales']].groupby('promo2_time_week').sum().reset_index()[df_2[['promo2_time_week','sales']].groupby('promo2_time_week').sum().reset_index()['promo2_time_week']>0].set_index('promo2_time_week').plot(kind='bar',
figsize=(15,5), color='black', rot=90, ax=axis[1], title='after promo');




In [ ]:
# Antes de a promoção começar - correlação

plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
sns.heatmap(df_2[['promo2_time_week','sales']].groupby('promo2_time_week').sum().reset_index()[df_2[['promo2_time_week','sales']].groupby('promo2_time_week').sum().reset_index()['promo2_time_week']<0].corr(),
           annot=True);

plt.subplot(1,2,2)
# Depois que a promoção começou - correlação

sns.heatmap(df_2[['promo2_time_week','sales']].groupby('promo2_time_week').sum().reset_index()[df_2[['promo2_time_week','sales']].groupby('promo2_time_week').sum().reset_index()['promo2_time_week']>0].corr(),
           annot=True);

plt.tight_layout()

<font size="+1">Hipótese 5:</font>

<s><b><i>Lojas com mais dias de promoção deveriam vender mais.</i></b></s>

Essa hipótese será validada no próximo ciclo do CRISP desse projeto.



<font size="+1">Hipótese 6:</font>

<b><i>Lojas com mais promoções consecutivas deveriam vender mais.</i></b>

Análise do gráfico de barras abaixo mostra que o maior volume de vendas é correspondente a promo, sem que acha uma continuidade com promo2. O gráfico de linhas abaixo mostra que ao longo do tempo tanto o perfil quanto a quantidade de vendas da promo é semelhante as vendas de promo+promo2

Portanto essa afirmação é <font color="red"><b>falsa</b></font>.

Relevância: pouco relevante.



In [ ]:
df_2.loc[:,['promo','promo2','sales']].groupby(['promo','promo2']).sum().plot(kind='bar', figsize=(15,5));

In [ ]:
# Comparação efeito nas vendas da segunda promoção quando se faz ou não a primeira


plt.figure(figsize=(15,5))

#Somente promo2 (0,1)
sns.lineplot(
    data=df_2[(df_2['promo']==0) & (df_2['promo2']==1)][['year_week','sales']].groupby('year_week').sum().reset_index(),
    y='sales',
    x='year_week')


# promo2 seguida da promo (1,1)
sns.lineplot(
    data=df_2[(df_2['promo']==1) & (df_2['promo2']==1)][['year_week','sales']].groupby('year_week').sum().reset_index(),
    y='sales',
    x='year_week')


# promo sem promo2 (1,0)
sns.lineplot(
    data=df_2[(df_2['promo']==1) & (df_2['promo2']==0)][['year_week','sales']].groupby('year_week').sum().reset_index(),
    y='sales',
    x='year_week')

plt.xticks(rotation=90)


plt.legend(labels=['Somente promo2','promo2 com promo','somente promo']);

<font size="+2">- Entidade <i>Temporal</i> :</font>

<font size="+1">Hipótese 7:</font>

<b><i>Lojas abertas durante o feriado de Natal deveriam vender mais..</i></b>

Análise abaixo mostra que, na primeira figura do painel, no geral a a soma de vendas de feriados públicos é maior. Esse comportamento é razoável pois essas vendas são referentes a todos os feriados (que não são Páscoa e nem Natal) somados. O segundo painel mostra que esse comportamento se mantém ao longo dos anos avaliados. O Natal de 2015 não foi incluído porque o ano ainda não estava fechado no dataset avaliado.

Já o terceiro painel mostra a média de vendas. Embora o valor absoluto de vendas no Natal seja menor, o Natal é o feriado que possui a maior média de vendas dentro do período avaliado

Portanto essa afirmação é <font color="red"><b>falsa</b></font>.

Relevância: Relevante.



In [ ]:

# Vendas nos feriados (foi removido 'regular_day' para facilitar a comparação)

plt.figure(figsize=(15,15))

plt.subplot(3,1,1)
sns.barplot(data=df_2.loc[df_2['state_holiday']!='regular day',['state_holiday','sales']].groupby('state_holiday').sum().reset_index(),
           x='state_holiday',
           y='sales');


# Vendas nos feriados ao longo dos anos

plt.subplot(3,1,2)
sns.barplot(data=df_2.loc[df_2['state_holiday']!='regular day',['state_holiday','year','sales']].groupby(['year','state_holiday']).sum().reset_index(),
           x='year',
           y='sales', hue='state_holiday');


# Média de Vendas nos feriados ao longo dos anos

plt.subplot(3,1,3)
sns.barplot(data=df_2.loc[df_2['state_holiday']!='regular day',['state_holiday','year','sales']].groupby(['year','state_holiday']).mean().reset_index(),
           x='year',
           y='sales', hue='state_holiday');

<font size="+1">Hipótese 8:</font>

<b><i>Lojas deveriam vender mais ao longo dos anos.</i></b>

Análise abaixo mostra que, para o período avaliado houve uma tendência de queda das vendas. Embora o ano de 2015 não seja um ano fechado, a observação leva a crer que a tendência será mantida no fechamento do ano.

Portanto essa afirmação é <font color="red"><b>falsa</b></font>.

Relevância: Relevante.



In [ ]:
plt.figure(figsize=(15,5))

sns.barplot(data=df_2[['year','sales']].groupby('year').sum().reset_index(),
            x='year',
            y='sales');

<font size="+1">Hipótese 9:</font>

<b><i>Lojas deveriam vender mais no segundo semestre do ano.</i></b>

Análise abaixo mostra que, do sexto mês do ano em diante existe uma tendência de queda nas vendas (painel da esquerda). A análise de correlação (painel direita) mostra um valor de -0.75 sugerindo uma correlação forte entre o aumento dos meses com a queda das vendas.

O painel abaixo mostra os meses para cada ano, entretanto não foi possível obter um padrão evidente.

Portanto essa afirmação é <font color="red"><b>falsa</b></font>.

Relevância: Relevante.



In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(2,2,1)
sns.barplot(data=df_2[['month','sales']].groupby(['month']).sum().reset_index(),
            x='month',
            y='sales');

plt.subplot(2,2,2)
sns.heatmap(data=df_2[['month','sales']].groupby(['month']).sum().reset_index().corr(),
           annot=True);

plt.subplot(2,1,2)
sns.barplot(data=df_2[['month','year','sales']].groupby(['month','year']).sum().reset_index(),
            x='month',
            y='sales',
           hue='year');

<font size="+1">Hipótese 10:</font>

<b><i>Lojas deveriam vender mais depois do dia 10 de cada mês.</i></b>

Análise abaixo mostra que, o total de vendas depois do dia 10 é maior do que o total de vendas antes do dia 10 (painel superior).

Ao desmembrar as velas antes e depois do dia 10 por cada ano avaliado o comportamento se mantém embora a diferença seja mais sutil.

Portanto essa afirmação é <font color="red"><b>falsa</b></font>.

Relevância: Relevante.



In [ ]:
aux = df_2[['day','sales']].groupby(['day']).sum().reset_index()

aux['before_after'] = aux['day'].apply(lambda x:'before_10th' if x<10 else 'after_10th')

aux_1 = df_2[['day','year','sales']].groupby(['day','year']).sum().reset_index()

aux_1['before_after'] = aux_1['day'].apply(lambda x:'before_10th' if x<10 else 'after_10th')

plt.figure(figsize=(15,10))



sns.barplot(data=aux[['before_after','sales']].groupby('before_after').sum().reset_index(),
            x='before_after',
            y='sales',
           order=aux[['before_after','sales']].groupby('before_after').sum().reset_index().sort_values(by='sales')['before_after']);


<font size="+1">Hipótese 11:</font>

<b><i>Lojas deveriam vender menos aos finais de semana.</i></b>

Análise abaixo mostra que, aos finais de semana a quantidade de vendas diminui.

Portanto essa afirmação é <font color="green"><b>verdadeira</b></font>.

Relevância: Relevante.



In [ ]:

plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
sns.barplot(data=df_2[['day_of_week','sales']].groupby('day_of_week').sum().reset_index(),
            x='day_of_week',
            y='sales');


aux = df_2[['day_of_week','sales']].groupby('day_of_week').sum().reset_index()

aux['weekday'] = aux['day_of_week'].apply(lambda x: 'weekdays' if x<6 else 'weekends')

plt.subplot(1,2,2)
sns.barplot(data=aux[['weekday','sales']].groupby('weekday').sum().reset_index(),
            x='weekday',
            y='sales');

<font size="+1">Hipótese 12:</font>

<b><i>Lojas deveriam vender menos durante os feriados escolares..</i></b>

Análise abaixo mostra que, no geral, nos feriados escolares a venda é menor quando comparada a períodos sem feriados escolares. Uma ressalva fica para a análise mês a mês (painel da direita) que mostra uma crescente nas vendas no mês 7 e no mês 8 as vendas nos dias de feriado escolar superando os demais dias, possívelmente o comportamento destoante desses meses é explicado pelo período de férias escolares.

Portanto essa afirmação é <font color="green"><b>verdadeira</b></font>.

Relevância: Relevante.



In [ ]:
plt.figure(figsize=(15,7))


plt.subplot(2,2,1)
sns.barplot(data=df_2[['school_holiday', 'sales']].groupby('school_holiday').sum().reset_index(),
            x='school_holiday',
            y='sales');

plt.subplot(2,2,3)
sns.barplot(data=df_2[['school_holiday', 'year', 'sales']].groupby(['school_holiday','year']).sum().reset_index(),
            x='year',
            y='sales', hue='school_holiday');

plt.subplot(1,2,2)
sns.barplot(data=df_2[['school_holiday', 'month', 'sales']].groupby(['school_holiday','month']).sum().reset_index(),
            x='month',
            y='sales', hue='school_holiday');

In [ ]:
# Conclusões

tab = [['Hipótese', 'Conclusão', 'Relevância'],
      ['H1', 'False','Baixa'],
      ['H2', 'False','Relevante'],
      ['H3', 'False','Relevante'],
      ['H4', 'False','Não relevante'],
      ['H5', '-','-'],
      ['H6', 'False','Pouco relevante'],
      ['H7', 'False','Relevante'],
      ['H8', 'False','Relevante'],
      ['H9', 'False','Relevante'],
      ['H10', 'Verdadeira','Relevante'],
      ['H11', 'Verdadeira','Relevante'],
      ['H12', 'Verdadeira','Relevante'],]

print(tabulate(tab, headers='firstrow'))

## <font color="#808080">Análise multivariada</font>

* Heatmap de correlação das variáveis numéricas (dataset com os features derivados):

In [ ]:
# Heatmap de correlação das variáveis numéricas (dataset com os features derivados)


plt.figure(figsize=(20,15))

sns.heatmap(df_2.select_dtypes(include=['float64','int64']).corr(), annot=True);

* Heatmap de correlação das variáveis numéricas (dataset original):

In [ ]:
# Heatmap de correlação das variáveis numéricas (dataset original)

plt.figure(figsize=(20,15))

sns.heatmap(num_attrib.corr(), annot=True);

* Heatmap de correlação das variáveis categóricas:

In [ ]:
# Heatmap de correlação das variáveis categóricas

plt.figure(figsize=(15,5))
plot_corr(df_2, ['state_holiday','store_type','assortment'] );

# Data preparation

## <font color="#808080">Train/Test split</font>

O split deve ser feito antes da preparação dos dados para evitar data leekage.

In [ ]:
df_3 = df_2.copy()

In [ ]:
# Train test split

# Dados temporais: split é feito em passado (treino) futuro (teste)
# como o modelo será construído para fazer previsão de 6 dias o conjunto será dividido com os 6
# semanas dias do dataset

df_3[['store','date']].groupby('store').min() # verificando a menor data para cada store no dataset

df_3[['store', 'date']].groupby('store').max() # verificando a menor data para cada store no dataset

In [ ]:
df_3[['store', 'date']].groupby('store').max().reset_index()['date'][0] # Obtendo o primeiro valor da maior data


# Subtrair 6 semanas da maior data (método datetime.timedelta)

df_3[['store', 'date']].groupby('store').max().reset_index()['date'][0] - datetime.timedelta(days=7*6)

In [ ]:
# Separando em treino/teste

#train
X_train = df_3.loc[df_3['date']<'2015-06-19 00:00:00', df_3.columns!='sales']
y_train = df_3.loc[df_3['date']<'2015-06-19 00:00:00', 'sales']

#test
X_test = df_3.loc[df_3['date']>='2015-06-19 00:00:00', df_3.columns!='sales']
y_test = df_3.loc[df_3['date']>='2015-06-19 00:00:00', 'sales']

In [ ]:

print('\nInformações dos dados:\n')
print('Mínimo conjunto treino: {0}'.format(X_train['date'].min()))
print('Máximo conjunto treino: {0}'.format(X_train['date'].max()))
print('\n')
print('Mínimo conjunto teste: {0}'.format(X_test['date'].min()))
print('Máximo conjunto teste: {0}'.format(X_test['date'].max()))


## <font color="#808080">Scaling - variáveis numéricas</font>

In [ ]:
X_train.select_dtypes(include=['float64','int64']).head()

In [ ]:
# Atributos numéricos escolhidos para o scaling:


# competition_distance (Robust Scaler)
 
# year (MinMax Scaler)

# competition_time_month (Robust Scaler)
 
# promo2_time_week (MinMax Scaler)

In [ ]:

# Para decidir entre MinMaxScaler e RobustScaler os boxplots das variáveis foram inspecionados.
# para variáveis com outiliers escolheu-se o Robust Scaler e para as demais o MinMax Scaler


# Scaling com Robust Scaler ('competition_distance' e 'competition_time_month')

# 'competition_distance'

rs = RobustScaler()

X_train['competition_distance'] = rs.fit_transform(X_train[['competition_distance']].values)

pickle.dump(rs, open('/home/gustavo/repos/Rossmann/competion_distance_scaler.pkl', 'wb')) # Saving scalers

# 'competition_time_month'

rs = RobustScaler()

X_train['competition_time_month'] = rs.fit_transform(X_train[['competition_time_month']].values)

pickle.dump(rs, open('/home/gustavo/repos/Rossmann/competition_time_month_scaler.pkl', 'wb')) # Saving scalers




# Scaling com MinMax Scaler ('year' e 'promo2_time_week')

# 'year'

mms = MinMaxScaler()

X_train['year'] = mms.fit_transform(X_train[['year']].values)

pickle.dump(mms, open('/home/gustavo/repos/Rossmann/year_scaler.pkl', 'wb')) # Saving scalers

# 'promo2_time_week'

mms = MinMaxScaler()

X_train['promo2_time_week'] = mms.fit_transform(X_train[['promo2_time_week']].values)

pickle.dump(mms, open('/home/gustavo/repos/Rossmann/promo2_time_week_scaler.pkl', 'wb')) # Saving scalers

In [ ]:
# Fazendo o scaling para o X_test

# Scaling com Robust Scaler ('competition_distance' e 'competition_time_month')

X_test['competition_distance'], X_test['competition_time_month'] = [RobustScaler().fit_transform(X_test[[i]].values) for i in ['competition_distance', 'competition_time_month']]


# Scaling com MinMax Scaler ('year' e 'promo2_time_week')

X_test['year'], X_test['promo2_time_week'] = [MinMaxScaler().fit_transform(X_test[[i]].values) for i in ['year', 'promo2_time_week']]

## <font color="#808080">Encoding - variáveis categóricas</font>

In [ ]:
X_train.select_dtypes(exclude=['float64','int64','datetime64[ns]']).head()

In [ ]:
# state_holiday (One-Hot encoding)

X_train = pd.get_dummies(data=X_train, columns=['state_holiday'], prefix=['state_holiday'])




# store_type (Label encoder)

le = LabelEncoder()

X_train['store_type'] = le.fit_transform(X_train['store_type'])

pickle.dump(le, open('/home/gustavo/repos/Rossmann/store_type_encoder.pkl', 'wb')) # Saving encoders



# assortment (Ordinal Encoder)

assortment_encoding_map = {'basic': 1, 'extended':2, 'extra':3}

X_train['assortment'] = X_train['assortment'].map(assortment_encoding_map)

In [ ]:
X_train.head()

In [ ]:
# Encoding do conjunto teste

# state_holiday (One-Hot encoding)

X_test = pd.get_dummies(data=X_test, columns=['state_holiday'], prefix=['state_holiday'])




# store_type (Label encoder)


X_test['store_type'] = LabelEncoder().fit_transform(X_test['store_type'])



# assortment (Ordinal Encoder)

assortment_encoding_map = {'basic': 1, 'extended':2, 'extra':3}

X_test['assortment'] = X_test['assortment'].map(assortment_encoding_map)

In [ ]:
X_test.head()

## <font color="#808080">Transformação - variável alvo</font>

In [ ]:
# Distribuição da variável alvo antes da transformação

sns.distplot(y_train)

In [ ]:
# Transformação log da variável alvo

y_train = np.log1p(y_train)

In [ ]:
# Distribuição da variável alvo depois da transformação

sns.distplot(y_train)

In [ ]:
# Conjunto teste antes da transformação

sns.distplot(y_test)

In [ ]:
# Transformação log da variável alvo - Conjunto teste

y_test = np.log1p(y_test)

In [ ]:
# Distribuição da variável alvo depois da transformação - Conjunto teste

sns.distplot(y_test)

## <font color="#808080">Transformação de natureza - variáveis de natureza cíclica</font>

In [ ]:
# Atributos de natureza cíclica:

# day_of_week

# month

# day

# week_of_year

In [ ]:
# Transformando os atributos

# day

X_train['day_sin'] = X_train['day'].apply(lambda x: np.sin(x*(2.*np.pi/30))) 
X_train['day_cos'] = X_train['day'].apply(lambda x: np.cos(x*(2.*np.pi/30)))

# day_of_week

X_train['day_of_week_sin'] = X_train['day_of_week'].apply(lambda x: np.sin(x*(2.*np.pi/7)))
X_train['day_of_week_cos'] = X_train['day_of_week'].apply(lambda x: np.cos(x*(2.*np.pi/7)))


# month

X_train['month_sin'] = X_train['month'].apply(lambda x: np.sin(x*(2.*np.pi/12)))
X_train['month_cos'] = X_train['month'].apply(lambda x: np.cos(x*(2.*np.pi/12)))


# week_of_year

X_train['week_of_year_sin'] = X_train['week_of_year'].apply(lambda x: np.sin(x*(2.*np.pi/52)))
X_train['week_of_year_cos'] = X_train['week_of_year'].apply(lambda x: np.cos(x*(2.*np.pi/52)))

In [ ]:
X_train.head()

In [ ]:
# Transformando os atributos - Conjunto teste

# day

X_test['day_sin'] = X_test['day'].apply(lambda x: np.sin(x*(2.*np.pi/30))) 
X_test['day_cos'] = X_test['day'].apply(lambda x: np.cos(x*(2.*np.pi/30)))

# day_of_week

X_test['day_of_week_sin'] = X_test['day_of_week'].apply(lambda x: np.sin(x*(2.*np.pi/7)))
X_test['day_of_week_cos'] = X_test['day_of_week'].apply(lambda x: np.cos(x*(2.*np.pi/7)))


# month

X_test['month_sin'] = X_test['month'].apply(lambda x: np.sin(x*(2.*np.pi/12)))
X_test['month_cos'] = X_test['month'].apply(lambda x: np.cos(x*(2.*np.pi/12)))


# week_of_year

X_test['week_of_year_sin'] = X_test['week_of_year'].apply(lambda x: np.sin(x*(2.*np.pi/52)))
X_test['week_of_year_cos'] = X_test['week_of_year'].apply(lambda x: np.cos(x*(2.*np.pi/52)))

In [ ]:
X_test.head()

# Feature selection

In [ ]:
# Train

X_train_1 = X_train.copy()

y_train_1 = y_train.copy()


# Test

X_test_1 = X_test.copy()

y_test_1 = y_test.copy()

In [ ]:
# Deletar variáveis que foram utilizadas para derivar outras variáveis à partir delas

X_train_1 = X_train_1.drop(columns=['week_of_year',
                                    'month',
                                    'day_of_week',
                                    'day',
                                    'promo2_since',
                                    'competition_since',
                                    'year_week'])

X_test_1 = X_test_1.drop(columns=['week_of_year',
                                    'month',
                                    'day_of_week',
                                    'day',
                                    'promo2_since',
                                    'competition_since',
                                    'year_week'])



## <font color="#808080">Algoritmo Boruta</font>

In [ ]:
# Converting X and y train to np array

X_train_1_np = X_train_1.drop(columns=['date']).values
y_train_1_np = y_train_1.values


# Instantiating Boruta

boruta = BorutaPy(RandomForestRegressor(n_jobs=-1),
                  n_estimators='auto',
                  verbose=2,random_state=42).fit(X_train_1_np, y_train_1_np)


In [ ]:
boruta.support_.tolist()

## <font color="#808080">Columns selected by Boruta</font>

In [ ]:
cols_to_select = [True,
                  True,
                  False,
                  True,
                  True,
                  True,
                  True,
                  True,
                  False,
                  True,
                  True,
                  False,
                  False,
                  True,
                  True,
                  False,
                  False,
                  False,
                  False,
                  True,
                  True,
                  True,
                  True,
                  False,
                  True,
                  False,
                  True]

In [ ]:
# X Train with Boruta selected features

X_train_1_boruta = X_train_1.drop(columns=['date'])

cols_selected_boruta = X_train_1_boruta.loc[:,cols_to_select].columns

print('\nColumns selected by Boruta:\n{x}\n'.format(x=list(cols_selected_boruta)))


print('\nColumns not selected by Boruta:\n{x}'.format(x=list(set(X_train_1.drop(columns=['date']).columns).difference(set(cols_selected_boruta)))))


In [ ]:
# Columns selected for further analysis

cols_selected = ['store', 
                 'promo', 
                 'store_type', 
                 'assortment', 
                 'competition_distance', 
                 'competition_open_since_month', 
                 'competition_open_since_year', 
                 'promo2_since_week', 
                 'promo2_since_year', 
                 'competition_time_month', 
                 'promo2_time_week', 
                 'day_sin', 
                 'day_cos', 
                 'day_of_week_sin', 
                 'day_of_week_cos',
                 'month_sin',
                 'month_cos',
                 'week_of_year_sin',
                 'week_of_year_cos']



# Machine learning

In [ ]:
# defining datasets with selected features

#train
X_train_boruta = X_train_1[cols_selected].copy()

#test

X_test_boruta = X_test_1[cols_selected].copy()


## <font color="#808080">Average model</font>

In [ ]:
aux = X_train_boruta.copy()
aux2= X_test_boruta.copy()
aux['sales'] = y_train_1
aux2['sales'] = y_test_1

# predictions (médias foram obtidas com os dados do conjunto de treino)
aux1 = aux[['store','sales']].groupby('store').mean().reset_index().rename(columns={'sales':'predictions'})

aux = pd.merge(aux2, aux1, on='store', how='left') # merging predictions with original dataset


# performance (comparação: médias obtidas do conjunto de treino com o y do conjunto de teste)

avg_metrics = ml_error( 'Average model', np.expm1(y_test_1), np.expm1(aux['predictions']))

avg_metrics

## <font color="#808080">Linear regression model</font>

In [ ]:
# Instantiating the model and making predictions

lr = LinearRegression().fit(X=X_train_boruta, y=y_train_1)

y_pred_lr = lr.predict(X_test_boruta)

# performance

lr_metrics = ml_error( 'Linear Regression model', np.expm1(y_test_1), np.expm1(y_pred_lr))

lr_metrics

## <font color="#808080">Lasso model</font>

In [ ]:
lasso = Lasso().fit(X=X_train_boruta, y=y_train_1)

y_predict_lasso = lasso.predict(X_test_boruta)

lasso_metrics = ml_error('Lasso model', np.expm1(y_test_1),np.expm1(y_predict_lasso))

lasso_metrics

## <font color="#808080">Random Forest</font>

In [ ]:
rf = RandomForestRegressor(n_jobs=-1, random_state=42).fit(X=X_train_boruta, y=y_train_1)

y_predict_rf = rf.predict(X_test_boruta)

rf_metrics = ml_error('Random Forest model', np.expm1(y_test_1),np.expm1(y_predict_rf))

rf_metrics

## <font color="#808080">XG Boost</font>

In [ ]:
xg_boost = xgb.XGBRegressor(objective='reg:squarederror',
                            n_estimators=100,
                            eta=0.01,
                            max_depth=10,
                            subsample=0.7,
                            colsample_bytree=0.9).fit(X=X_train_boruta, y=y_train_1)

y_predict_xg_boost = xg_boost.predict(X_test_boruta)

xg_boost_metrics = ml_error('XGboost model', np.expm1(y_test_1),np.expm1(y_predict_xg_boost))

xg_boost_metrics                            

## <font color="#808080">Comparing metrics</font>

In [ ]:
model_results = pd.concat([avg_metrics, lr_metrics, lasso_metrics, rf_metrics, xg_boost_metrics]).sort_values('RMSE')
model_results.index = range(len(model_results))

model_results

## <font color="#808080">Cross Validation</font>

In [ ]:
# Columns to select for date separation in cross validation

cols_selected = ['store', 
                 'promo', 
                 'store_type', 
                 'assortment', 
                 'competition_distance', 
                 'competition_open_since_month', 
                 'competition_open_since_year', 
                 'promo2_since_week', 
                 'promo2_since_year', 
                 'competition_time_month', 
                 'promo2_time_week', 
                 'day_sin', 
                 'day_cos', 
                 'day_of_week_sin', 
                 'day_of_week_cos',
                 'month_sin',
                 'month_cos',
                 'week_of_year_sin',
                 'week_of_year_cos']

# add 'date' and 'sales'
columns_to_add = ['date']


# final coluns
cols_total = cols_selected.copy()
cols_total.extend(columns_to_add)


# X train for Cross Validation
X_train_CrossVal = X_train_1[cols_total]


In [ ]:
# Linear Regression - Cross Validation Metrics

lr_CV_metrics = cross_validation_Time_Series(5, X_train_CrossVal, y_train, 
                                             LinearRegression(), 'Linear Regression')

lr_CV_metrics

In [ ]:
# Lasso - Cross Validation Metrics

lasso_CV_metrics = cross_validation_Time_Series(5, X_train_CrossVal, y_train,
                                                Lasso(), 'Lasso')

lasso_CV_metrics

In [ ]:
# Random Forest - Cross Validation Metrics

rf_CV_metrics = cross_validation_Time_Series(5, X_train_CrossVal, y_train, 
                                             RandomForestRegressor(n_jobs=-1, random_state=42), 
                                             'Random Forest',
                                             verbose=True)

rf_CV_metrics

In [ ]:
# XGBoost - Cross Validation Metrics

xgboost_CV_metrics = cross_validation_Time_Series(5, X_train_CrossVal, y_train, 
                                             xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100,  eta=0.01, max_depth=10, subsample=0.7, colsample_bytree=0.9), 
                                             'XGBoost',
                                             verbose=True)

xgboost_CV_metrics

In [ ]:
# Comparing results

cv_results = pd.concat([lr_CV_metrics,lasso_CV_metrics, rf_CV_metrics, xgboost_CV_metrics]).sort_values('RMSE CV')

cv_results.index = range(len(cv_results))

cv_results

# Hiperparameter fine tuning

## <font color="#808080">Random Search</font>

In [ ]:
# Random search implementation for XGBoost

parameters = {'n_estimators':[1500, 1700, 2500, 3000, 3500],
              'eta':[0.01, 0.03],
              'max_depth':[3, 5, 9],
              'subsample':[0.1, 0.5, 0.7],
              'colsample_bytree':[0.3, 0.7, 0.9],
              'min_child_weight':[3, 8, 15]} # choosing the interval values to evaluate 

max_evaluation = 10 # choosing the number o experiments to perform

k_fold = 5 # chosing k-fold for cross-validation

In [ ]:
# Implementing Random Search

df_metrics_hp = pd.DataFrame()

for i in range(max_evaluation):

  hp = {k: random.sample(v,1)[0] for k, v in parameters.items()} # choosing fixed key and random values

  print(hp)

  xg_boost = xgb.XGBRegressor(objective='reg:squarederror',
                              n_estimators=hp['n_estimators'],
                              eta=hp['eta'],
                              max_depth=hp['max_depth'],
                              subsample=hp['subsample'],
                              colsample_bytree=hp['colsample_bytree'],
                              min_child_weight=hp['min_child_weight'],
                              n_jobs=100)

  xgboost_hp_cv_metrics = cross_validation_Time_Series(k_fold, 
                                                       X_train_CrossVal, 
                                                       y_train, 
                                                       xg_boost, 
                                                       'XGBoost', 
                                                       verbose=True)
  
  df_metrics_hp = pd.concat([df_metrics_hp, xgboost_hp_cv_metrics ])

  print(df_metrics_hp)

df_metrics_hp  

## <font color="#808080">Final model</font>

In [ ]:
# Setting parameters

best_parameters = {'n_estimators': 1700, 
                   'eta': 0.03, 
                   'max_depth': 9, 
                   'subsample': 0.7, 
                   'colsample_bytree': 0.7, 
                   'min_child_weight': 3}

In [ ]:
# Training the model

xg_boost = xgb.XGBRegressor(objective='reg:squarederror',
                            n_estimators=best_parameters['n_estimators'],
                            eta=best_parameters['eta'],
                            max_depth=best_parameters['max_depth'],
                            subsample=best_parameters['subsample'],
                            colsample_bytree=best_parameters['colsample_bytree'],
                            min_child_weight=best_parameters['min_child_weight']).fit(X=X_train_boruta,
                                                                                      y=y_train_1)

y_predict_xg_boost = xg_boost.predict(X_test_boruta) #predictions on test dataset

xg_boost_metrics = ml_error('XGboost model', np.expm1(y_test_1),np.expm1(y_predict_xg_boost))

xg_boost_metrics  

In [ ]:
# Calculating MPE (Mean Percentage Error)

xg_boost_metrics['MPE'] = mean_percentage_error(np.expm1(y_test_1), np.expm1(y_predict_xg_boost))

xg_boost_metrics

In [ ]:
# Saving trained model

pickle.dump(xg_boost, open('/home/gustavo/repos/Rossmann/model_XGBoost_tuned.pkl', 'wb'))

# Error interpretation

## <font color="#808080">Main</font>

In [ ]:
df_4 = X_test_1[cols_total].copy()

df_4.head()

In [ ]:
# Adding 'sales' column to df_4

df_4 = pd.concat([df_4,y_test], axis=1)

df_4.head(3)

In [ ]:
# Adding 'predictions' column

df_4['predictions'] = y_predict_xg_boost

df_4.head(3)

In [ ]:
# Rescaling 'sales' and 'predictions' columns to the original values

df_4[['sales','predictions']] = df_4[['sales','predictions']].apply(lambda x: np.expm1(x))

df_5 = df_4.copy()

df_4.head(3)

## <font color="#808080">Business performance</font>

In [ ]:
# Total sum of 'sales' and 'predictions' by store

df_4 = df_4[['store','sales','predictions']].groupby('store').sum().reset_index()

print(df_4.shape)

df_4.head(5)

In [ ]:
# MAE 

df_4_MAE = df_4[['store','sales','predictions']].groupby('store').apply(lambda x: mean_absolute_error(x['sales'], x['predictions']))

# MAPE 

df_4_MAPE = df_4[['store','sales','predictions']].groupby('store').apply(lambda x: mean_absolute_percentage_error(x['sales'], x['predictions']))


# Concat df_4_MAE and df_4_MAPE

df_aux = pd.concat([df_4_MAE,df_4_MAPE], axis=1).reset_index().rename(columns={0:'MAE', 1:'MAPE'})


# Merging df_4 and df_aux

df_4 = pd.merge(df_4, df_aux, how='inner', on='store')



print(df_4.shape)

df_4.head(4)

In [ ]:
# Creating 'worst_scenario' and 'best_scenario' columns

# 'worst_scenario' (predictions - MAE)

df_4['worst_scenario'] = df_4['predictions'] - df_4['MAE']

# 'best_scenario' (predictions + MAE)

df_4['best_scenario'] = df_4['predictions'] + df_4['MAE']

df_4[['store', 'predictions', 'worst_scenario', 'best_scenario', 'MAE', 'MAPE']]

In [ ]:
# Worst performing predictions by store

df_4[['store', 'predictions', 'worst_scenario', 'best_scenario', 'MAE', 'MAPE']].sort_values('MAPE', ascending=False)

In [ ]:
# MAPE by store number

plt.figure(figsize=(15,6))
sns.scatterplot(x='store', y='MAPE', data=df_4);

In [ ]:
# Agrupando as vendas por data e loja (para incluir 'best' and 'worst scenario' ao longo do tempo)


df_5_aux = df_5[['date','store', 'sales','predictions']].groupby(['date','store']).sum()

df_5_aux

In [ ]:
df_5_aux_1 = pd.DataFrame(df_5[['date','store', 'sales','predictions']].groupby(['date','store']).apply(lambda x: mean_absolute_error(x['sales'],x['predictions'])))


df_5_aux_1


In [ ]:
df_5_aux_2 = pd.concat([df_5_aux, df_5_aux_1], axis=1).rename(columns={0:'MAE'})

df_5_aux_2

In [ ]:
df_5_aux_a = pd.DataFrame(df_5[['date','store', 'sales','predictions']].groupby(['date','store']).apply(lambda x: mean_absolute_percentage_error(x['sales'],x['predictions'])))


df_5_aux_a

In [ ]:
df_5_aux_2 = pd.concat([df_5_aux_2, df_5_aux_a], axis=1).rename(columns={0:'MAPE'}).reset_index()

df_5_aux_2

In [ ]:
# Creating best_scenario and worst_scenario columns:

df_5_aux_2['best_scenario'] = df_5_aux_2['predictions'] + df_5_aux_2['MAE']

df_5_aux_2['worst_scenario'] = df_5_aux_2['predictions'] - df_5_aux_2['MAE']

df_5_aux_2

In [ ]:
df_5_aux_2[['date', 'store','predictions', 'best_scenario', 'worst_scenario', 'MAE', 'MAPE']]

## <font color="#808080">Total performance</font>

In [ ]:
# Performance for all stores

pd.DataFrame(pd.DataFrame(df_4[['predictions', 'worst_scenario', 'best_scenario', 'MAE']].sum()).apply(lambda x: '$ %.2f' % x, axis=1)).rename(columns={0:'Total'})


## <font color="#808080">Machine learning performance</font>

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(2,2,1)
sns.lineplot(x='date', y='sales', data=df_5, label='Sales')
sns.lineplot(x='date', y='predictions', data=df_5, color='orange', label='Predictions')
plt.xticks(rotation=45)

plt.subplot(2,2,3)
sns.lineplot(x='date', y='predictions', data=df_5_aux_2, color='orange', label='Predictions')
sns.lineplot(x='date', y='best_scenario', data=df_5_aux_2, color='black', linestyle='--', label='MAE Upper and lower limits')
sns.lineplot(x='date', y='worst_scenario', data=df_5_aux_2, color='black', linestyle='--')
plt.xticks(rotation=45)

plt.subplot(2,2,2)
sns.distplot(y_test_1 - y_predict_xg_boost, color='orange')
plt.xlabel('Residuals')

plt.subplot(2,4,7)
sns.regplot(x=df_5['predictions'], y=df_5['sales'], 
            ci=95, 
            scatter_kws={"color": "grey"}, line_kws={"color": "black"})


plt.subplot(2,4,8)
sns.regplot(x=df_5['predictions'], y=(df_5['sales'] - df_5['predictions']), 
            ci=95, 
            scatter_kws={"color": "orange"}, line_kws={"color": "black"})

plt.ylabel('Residuals')

plt.tight_layout();

# Model deployment

## <font color="#808080">Class Rossmann - Data_prep.py</font> 

O código abaixo deverá ser posteriormente colocado em um script .py

A classe criada abaixo será responsável por fazer a limpeza dos dados.

Apagar do código abaixo tudo relacionado a:'sales', 'costumer', pois esses dados não estarão disponíveis em produção. Lembrar também de colocar no início do código o import de todas as bibliotecas necessárias.

In [26]:
import pickle
import pandas as pd
import numpy as np
import inflection
import datetime
import json

class Data_Prep(object):
    
    def __init__(self):
        self.competition_distance_scaler   = pickle.load(open('/home/gustavo/repos/Rossmann/competion_distance_scaler.pkl', 'rb')) # Opening scaler
        self.competition_time_month_scaler = pickle.load(open('/home/gustavo/repos/Rossmann/competition_time_month_scaler.pkl', 'rb')) # Opening scaler
        self.year_scaler                   = pickle.load(open('/home/gustavo/repos/Rossmann/year_scaler.pkl', 'rb')) # Opening scaler
        self.promo2_time_week_scaler       = pickle.load(open('/home/gustavo/repos/Rossmann/promo2_time_week_scaler.pkl', 'rb')) # Opening scalers
        self.store_type_encoder            = pickle.load(open('/home/gustavo/repos/Rossmann/store_type_encoder.pkl', 'rb')) # Opening encoders
        
        self.df_store                      = pd.read_csv('/home/gustavo/repos/Rossmann/store.csv')
    
    
    
    def data_cleaning(self, df):
        
        # Merging df_test and df_store:
        df = pd.merge(df, self.df_store, on='Store', how='left')
        
        # Dropping 'Id' column from df_test dataset:
        df = df.drop(columns='Id')
        


        # Changing column 'Date' to datetime

        df['Date'] = pd.to_datetime(df['Date'])

        # Adjusting column names

        df.columns = list(map(lambda x: inflection.underscore(x), df.columns)) #changing to underscore + lower(snakecase)



        # Filling NA's on 'competition_distance' column ===========================================


        ## LÓGICA USADA: se a distância está NA assumiu-se que a distância da loja concorrente é muita alta.
        ### então foi feito o preenchimento por um valor acima da distância mais alta do dataset

        df['competition_distance'] = df['competition_distance'].apply(lambda x: 200000.0 if pd.isna(x) else x)



        # Filling NA's on 'competition_open_since_month' column ==================================


        ## LÓGICA USADA: completou-se o valor NA usando o mês da coluna 'date' (data da venda) da respectiva linha

        df['competition_open_since_month'] = df[['date','competition_open_since_month']].apply(lambda x: x['date'].month if pd.isna(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)



        # Filling NA's on 'competition_open_since_year' column =====================================


        ## LÓGICA USADA: completou-se o valor NA usando o ano da coluna 'date' (data da venda) da respectiva linha

        df['competition_open_since_year'] = df[['date','competition_open_since_year']].apply(lambda x: x['date'].year if pd.isna(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)



        # Filling NA's on 'promo2_since_week' column =================================================

        ## LÓGICA USADA: todos os valores onde a coluna promo2 estavam 0 (não houve promoção na loja) a coluna
        ### 'promo2_since_week' estava NA, então assumiu-se que não houve promoção nessa loja e preencheu-se com 0

        df['promo2_since_week'] = df['promo2_since_week'].fillna(0)



        # Filling NA's on 'promo2_since_year' column ===================================================

        ## LÓGICA USADA: todos os valores onde a coluna promo2 estavam 0 (não houve promoção na loja) a coluna
        ### 'promo2_since_year' estava NA, então assumiu-se que não houve promoção nessa loja e preencheu-se com 0

        df['promo2_since_year'] = df['promo2_since_year'].fillna(0)




        # Filling NA's on 'promo_interval' column ========================================================

        ## LÓGICA USADA: todos os valores onde a coluna promo2 estavam 0 (não houve promoção na loja) a coluna
        ### 'promo_interval' estava NA, então assumiu-se que não houve promoção nessa loja e preencheu-se com 0

        df['promo_interval'] = df['promo_interval'].fillna(0)



        # Abaixo será criado um dicionário que auxiliará na conversão dos meses de números em letras:

        # criando mapeamento dos meses
        month_map = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}


        # criado a coluna 'month_map'
        ## LÒGICA: extraíu-se o mês (em número) da coluna 'date' e converteu-se em mês (em palavra) colocando em uma nova
        ### coluna

        df['month_map'] = df['date'].dt.month.map(month_map)


        # criando a coluna 'is_promo'
        # LÓGICA: Se o dia da venda (coluna 'data') estiver em um mês onde foi feita a promoção (coluna 'promo_interval')
        ## então será atribuído 1 na nova coluna 'is_promo', se não tiver promoção ou a venda tiver sido feita em um mês
        ## fora da promoção então o valor atribuído será 0

        df['is_promo'] = df[['month_map', 'promo_interval']].apply(lambda x: 0 if x['promo_interval']==0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)


        # Changing data types after previous columns creation

        df[['competition_open_since_month',
            'competition_open_since_year',
            'promo2_since_week',
            'promo2_since_year']] = df[['competition_open_since_month',
                                        'competition_open_since_year',
                                        'promo2_since_week',
                                        'promo2_since_year']].astype(int)
        
        return df
    
    
    def feature_engineering(self, df_1):    
    
    
        # Changing column 'assortment'

        df_1['assortment'] = df_1['assortment'].apply(lambda x:'basic' if x=='a' else 'extra' if x=='b' else 'extended')


        # Changing column 'state_holiday'

        df_1['state_holiday'] = df_1['state_holiday'].apply(lambda x:'Public holiday' if x=='a' else 'Easter' if x=='b' else 'Christmas' if x=='c' else 'regular day')


        # Derivando novas variáveis:


        # year

        df_1['year'] = df_1['date'].dt.year


        # month

        df_1['month'] = df_1['date'].dt.month


        # day

        df_1['day'] = df_1['date'].dt.day


        # week of year

        df_1['week_of_year'] = df_1['date'].dt.weekofyear


        # year week

        df_1['year_week'] = df_1['date'].dt.strftime('%Y-%W') # esse comando apenas coleta a data da venda e muda a formatação




        # competition_since (quanto tempo desde o início da competição ate a data da compra)

        # 1o passo - juntar as colunas 'competition_open_since_month' com 'competition_open_since_year'

        df_1['competition_since'] = df_1[['competition_open_since_year','competition_open_since_month']].apply(lambda x: datetime.datetime(year= x['competition_open_since_year'], month=x['competition_open_since_month'], day=1), axis=1)

        # 2o passo - subtrair as datas de 'date' e 'competition_since' e dividir 30 (granularidade mês)
        df_1['competition_time_month'] = ((df_1['date'] - df_1['competition_since'])/ 30).apply(lambda x: x.days).astype(int)




        # promo2_since (quanto tempo desde a a promoção estar ativa até a data da compra)

        # 1° passo: juntar a coluna 'promo2_since_year' com a coluna 'promo2_since_week' e armazenar em uma nova coluna
        df_1['promo2_since'] = df_1.apply(lambda x: str(x['promo2_since_year']) + '-' + str(x['promo2_since_week']),axis=1)

        # 2° passo: converter 'promo_since' de string para data
        # OBSERVAÇÃO IMPORTANTE: tive que fazer uma adaptação no código.
        # no código original a atribuição é feita sobre todas as linhas da coluna 'promo_since', no entanto o lambda
        # não estava funcionando na coluna 'promo_since' nas linhas com valores zerados '0-0' (proveniente dos valores
        # NA's que foram substituídos por 0 na etapa de preenchimento de NA's) então tive que aplicar a função no df
        # filtrado (df_1['promo_since']!='0-0')
        df_1.loc[df_1['promo2_since']!='0-0','promo2_since'] = df_1.loc[df_1['promo2_since']!='0-0','promo2_since'].apply(lambda x: datetime.datetime.strptime(x+'-1', '%Y-%W-%w') - datetime.timedelta(days=7) )

        # 3° passo: subtrair a data da coluna 'date' da coluna 'promo_since' (criada no passo acima), mantendo o filtro
        # condicional de linhas (df_1['promo_since']!='0-0') a atribuindo a uma nova coluna 'promo_time_week' mantendo
        # o filtro de linhas
        df_1.loc[df_1['promo2_since']!='0-0','promo2_time_week'] = ((df_1.loc[df_1['promo2_since']!='0-0','date'] - df_1.loc[df_1['promo2_since']!='0-0','promo2_since']) / 7).apply(lambda x: x.days).astype(int)

        # 4° passo: os filtros de linha anteriores resultaram apenas em dados preenchidos nas linhas do filtro. As linhas
        # não pertencentes ao filtro foram preenchidas com NA's. Preencher esses valores com 0
        df_1['promo2_time_week'] = df_1['promo2_time_week'].fillna(0)
        
        
        
        # Quando a loja está fechada ('open'==0) a quantidade de vendas e clientes é 0. Eliminar essas linhas.
        # eliminar também as linhas onde não houve vendas (df_2['sales']==0)

        df_1 = df_1[df_1['open']!=0]

        # Eliminar a coluna 'customers' (no momento do modelo em produção a informação sobre a quantidade
        # de clientes na loja não estará disponível).
        # coluna 'open' estará com valor constante (1 - loja aberta)
        # demais colunas são auxiliares

        df_1 = df_1.drop(['open','promo_interval','month_map'], axis=1)      
      
        
        
        return df_1
    
    
    
    def data_preparation(self, df_1):
        
        #train
        X_train = df_1


        # Atributos numéricos escolhidos para o scaling:


        # competition_distance (Robust Scaler)

        # year (MinMax Scaler)

        # competition_time_month (Robust Scaler)

        # promo2_time_week (MinMax Scaler)


        # Para decidir entre MinMaxScaler e RobustScaler os boxplots das variáveis foram inspecionados.
        # para variáveis com outiliers escolheu-se o Robust Scaler e para as demais o MinMax Scaler


        # Scaling com Robust Scaler ('competition_distance' e 'competition_time_month')

        # 'competition_distance'
        
        X_train['competition_distance'] = self.competition_distance_scaler.transform(X_train[['competition_distance']].values)

        

        # 'competition_time_month'

        X_train['competition_time_month'] = self.competition_time_month_scaler.transform(X_train[['competition_time_month']].values)

        




        # Scaling com MinMax Scaler ('year' e 'promo2_time_week')

        # 'year'

        X_train['year'] = self.year_scaler.transform(X_train[['year']].values)

        

        # 'promo2_time_week'

        X_train['promo2_time_week'] = self.promo2_time_week_scaler.transform(X_train[['promo2_time_week']].values)

        


        ## <font color="#808080">Encoding - variáveis categóricas</font>


        # state_holiday (One-Hot encoding)

        X_train = pd.get_dummies(data=X_train, columns=['state_holiday'], prefix=['state_holiday'])




        # store_type (Label encoder)

        X_train['store_type'] = self.store_type_encoder.transform(X_train['store_type'])




        # assortment (Ordinal Encoder)

        assortment_encoding_map = {'basic': 1, 'extended':2, 'extra':3}

        X_train['assortment'] = X_train['assortment'].map(assortment_encoding_map)





        ## <font color="#808080">Transformação de natureza - variáveis de natureza cíclica</font>

        # Atributos de natureza cíclica:

        # day_of_week

        # month

        # day

        # week_of_year

        # Transformando os atributos

        # day

        X_train['day_sin'] = X_train['day'].apply(lambda x: np.sin(x*(2.*np.pi/30))) 
        X_train['day_cos'] = X_train['day'].apply(lambda x: np.cos(x*(2.*np.pi/30)))

        # day_of_week

        X_train['day_of_week_sin'] = X_train['day_of_week'].apply(lambda x: np.sin(x*(2.*np.pi/7)))
        X_train['day_of_week_cos'] = X_train['day_of_week'].apply(lambda x: np.cos(x*(2.*np.pi/7)))


        # month

        X_train['month_sin'] = X_train['month'].apply(lambda x: np.sin(x*(2.*np.pi/12)))
        X_train['month_cos'] = X_train['month'].apply(lambda x: np.cos(x*(2.*np.pi/12)))


        # week_of_year

        X_train['week_of_year_sin'] = X_train['week_of_year'].apply(lambda x: np.sin(x*(2.*np.pi/52)))
        X_train['week_of_year_cos'] = X_train['week_of_year'].apply(lambda x: np.cos(x*(2.*np.pi/52)))
        
        
        # Columns selected for further analysis

        cols_selected = ['store', 
                         'promo', 
                         'store_type', 
                         'assortment', 
                         'competition_distance', 
                         'competition_open_since_month', 
                         'competition_open_since_year', 
                         'promo2_since_week', 
                         'promo2_since_year', 
                         'competition_time_month', 
                         'promo2_time_week', 
                         'day_sin', 
                         'day_cos', 
                         'day_of_week_sin', 
                         'day_of_week_cos',
                         'month_sin',
                         'month_cos',
                         'week_of_year_sin',
                         'week_of_year_cos']
        
        
        
        return X_train[cols_selected]
    
    
    def get_predictions(self, model, test_raw, df_3):
        
        #prediction
        predictions = model.predict(df_3)
        
        #join predictions into the original data        
        test_raw.loc[df_3.index, 'Predictions'] = np.expm1(predictions)
        
        
   
        return test_raw.to_json(orient='records', date_format='iso')    
         



## <font color="#808080">Creating API Handler - API_handler.py</font>

O código abaixo também deverá ser posteriormente colocado em um script .py

In [22]:
# Library imports

import pickle
import pandas as pd
from flask             import Flask, request, Response
import json
from Rossmann_class.Data_Prep import Data_Prep # from diretório.nome_do_arquivo_classe import nome_da_classe

# loading model
model = pickle.load(open('/home/gustavo/repos/Rossmann/model_XGBoost_tuned.pkl', 'rb'))

# initialize API
app = Flask(__name__)

# criando o endpoint (url que receberá os dados):
@app.route( '/rossmann/predict', methods=['POST'] )


# Sempre após receber uma chamada o endpoint executa alguma função. Para esse caso ele executará a função 
## definida abaixo:

def rossmann_predict():
    test_json = request.get_json() # os dados da requisição virão na forma de json.
    
   
    if test_json: # there is data
        test_json = json.loads(test_json)
        
        if isinstance( test_json, dict ): #testar se o dado é uma requisição de apenas 1 linha (virá como dict)
            test_raw = pd.DataFrame( test_json, index=[0] )
            
        else: # do contrário a requisição virá na forma de lista (com cada ítem sendo um dicionario):
            test_raw = pd.DataFrame( test_json, columns=test_json[0].keys() )
            
        
        # Instantiate Rossmann class
        pipeline = Data_Prep()
        
        # data cleaning
        df1 = pipeline.data_cleaning( test_raw )
        
        # feature engineering
        df2 = pipeline.feature_engineering( df1 )
        
        # data preparation
        df3 = pipeline.data_preparation( df2 )
        
        # prediction
        df_response = pipeline.get_predictions( model, test_raw, df3 )
        
        return df_response
        
        
    else: # se o dado não existir
        return Response( '{}', status=200, mimetype='application/json' )

if __name__ == '__main__':
    app.run( '0.0.0.0' )
    #      (local host)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.18.10:5000/ (Press CTRL+C to quit)


## <font color="#808080">Creating API tester</font>

In [53]:
# Loading test dataset

df_test = pd.read_csv('/home/gustavo/repos/Rossmann/test.csv')

print(df_test.shape)

df_test.head()

(41088, 8)


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [55]:
# Converting df to json

data_json = df_test.to_json(orient='records', date_format='iso')



# API Call =====================================================================================

# Para a call funcionar o servidor local tem que estar ativo (terminal: python API_handler.py)
# Se o resultado da execução dessa célula for 200 a requisição funcionou

url = 'http://0.0.0.0:5000/rossmann/predict' #endpoint (onde os dados serão enviados.0.0.0.0 significa que é um local host não conectado a internet. Porta 5000 é padrão do Flask)

header = {'Content-type':'application/json'} #indica para a API qual tipo de requisição se está fazendo





# Fazendo a requisição

response = requests.post(url=url, headers=header, json=data_json)

print('Status Code: {}'.format(response.status_code))





Status Code: 200


In [57]:
df_predictions = pd.DataFrame(response.json(), columns=response.json()[0].keys())

df_predictions

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Predictions
0,1,1,4,2015-09-17,1.0,1,0,0,4069.109619
1,2,3,4,2015-09-17,1.0,1,0,0,7472.016602
2,3,7,4,2015-09-17,1.0,1,0,0,8666.511719
3,4,8,4,2015-09-17,1.0,1,0,0,6503.457031
4,5,9,4,2015-09-17,1.0,1,0,0,6918.713867
...,...,...,...,...,...,...,...,...,...
41083,41084,1111,6,2015-08-01,1.0,0,0,0,2922.214600
41084,41085,1112,6,2015-08-01,1.0,0,0,0,7750.389648
41085,41086,1113,6,2015-08-01,1.0,0,0,0,6185.503906
41086,41087,1114,6,2015-08-01,1.0,0,0,0,21719.390625


# <font color="red">Rascunhos</font>

O histograma acima mostra para a variável 'promo' que o valor 0 é mais frequente do que o valor 1.

Entretanto é importante lembrar que o eixo y do histograma é uma contagem de frequências e não mostra o total valor vendido. Considerando que é mais comum não haver promoção do que quando há, o valor mostrado é razoavel

O boxplot abaixo mostra que a média de vendas (valor das vendas - coluna 'sales') é maior quando há promoção.

In [ ]:
sns.boxplot(x='promo', y='sales', data=df_2)

In [ ]:
# Promoção parece não fazer efeito

# Receita média das vendas sem promoção é maior do que em promoção

df_2[['is_promo','sales']].groupby('is_promo').mean()

In [ ]:
# Porém a mediana das vendas sem promoção é parecida do que em promoção

df_2[['is_promo','sales']].groupby('is_promo').median()

In [ ]:
# Promoção 2 parece não fazer efeito

# Média

df_2[['promo2','sales']].groupby('promo2').mean()

In [ ]:
# Mediana

df_2[['promo2','sales']].groupby('promo2').median()

In [20]:
# Testando o handler no snipet


# Loading model
model = pickle.load(open('/home/gustavo/repos/Rossmann/model_XGBoost_tuned.pkl', 'rb'))


if data_json:
    
    if isinstance(test_json, dict): # testar se o dado é uma requisição de apenas 1 linha (virá como dict)
        df_test = pd.DataFrame(test_json, index=[0])
        
    else: # do contrário a requisição virá na forma de lista (com cada ítem sendo um dicionario):
        #converter para df
        df_test = pd.DataFrame(data_json, columns=data_json[0].keys())
    
    



    dataprep = Data_Prep() # Instanciando o Data_Prep

    test_raw = df_test.copy()



    df_1 = dataprep.data_cleaning(df_test)

    df_2 = dataprep.feature_engineering(df_1)

    df_3 = dataprep.data_preparation(df_2)

    df_4 = dataprep.get_predictions(model, test_raw, df_3)


/tmp/ipykernel_3963/2182421447.py:156: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_1['week_of_year'] = df_1['date'].dt.weekofyear


In [303]:
# Testando o handler/flask no snipet


import pickle
import pandas as pd
from flask             import Flask, request, Response
from Rossmann_class.Data_Prep import Data_Prep # from diretório.nome_do_arquivo_classe import nome_da_classe

# loading model
model = pickle.load(open('/home/gustavo/repos/Rossmann/model_XGBoost_tuned.pkl', 'rb'))

# initialize API
app = Flask(__name__)

# criando o endpoint (url que receberá os dados):
@app.route( '/rossmann/predict', methods=['POST'] )


# Sempre após receber uma chamada o endpoint executa alguma função. Para esse caso ele executará a função 
## definida abaixo:

def rossmann_predict():
    test_json = request.get_json() # os dados da requisição virão na forma de json.
   
    if test_json: # there is data
        if isinstance( test_json, dict ): #testar se o dado é uma requisição de apenas 1 linha (virá como dict)
            test_raw = pd.DataFrame( test_json, index=[0] )
            
        else: # do contrário a requisição virá na forma de lista (com cada ítem sendo um dicionario):
            test_raw = pd.DataFrame( test_json, columns=test_json[0].keys() )
            
        
        # Instantiate Rossmann class
        pipeline = Data_Prep()
        
        # data cleaning
        df1 = pipeline.data_cleaning( test_raw )
        
        # feature engineering
        df2 = pipeline.feature_engineering( df1 )
        
        # data preparation
        df3 = pipeline.data_preparation( df2 )
        
        # prediction
        df_response = pipeline.get_prediction( model, test_raw, df3 )
        
        return df_response
        
        
    else: # se o dado não existir
        return Response( '{}', status=200, mimetype='application/json' )

if __name__ == '__main__':
    app.run( '0.0.0.0' )
    #      (local host)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.18.10:5000/ (Press CTRL+C to quit)


In [298]:
# Library imports

from Rossmann_class.Data_Prep import Data_Prep # from diretório.nome_do_arquivo_classe import nome_da_classe
from flask             import Flask, request, Response
import pandas as pd
import pickle

# Loading model

model = pickle.load(open('/home/gustavo/repos/Rossmann/model_XGBoost_tuned.pkl', 'rb'))


# Inicializando a API ====================================================================


# instantiating Flask:

app = Flask( __name__ )


# criando o endpoint (url que receberá os dados):

@app.route('/rossmann/predict', methods=['POST'])


# Sempre após receber uma chamada o endpoint executa alguma função. Para esse caso ele executará a função 
## definida abaixo:

def rossmann_predict():
    
    test_json = request.get_json() # os dados da requisição virão na forma de json.
    
    # Testar se o dado veio:
    
    if test_json: # se o dado da request existir:
        
        if isinstance(test_json, dict): # testar se o dado é uma requisição de apenas 1 linha (virá como dict)
            # converter json to df:
            test_raw = pd.DataFrame(test_json, index=[0])
            
        else: # do contrário a requisição virá na forma de lista (com cada ítem sendo um dicionario):
            # converter json to df:        
            test_raw = pd.DataFrame(test_json, columns=test_json[0].keys())
        
        
        
        # Após definir o dataframe, o passo seguinte é fazer as limpezas dos dados:
        
        # Instanciando a classe Rossmann
        
        pipeline = Data_Prep()
        
        # Aplicando o método de limpeza de dados
        df_1 = pipeline.data_cleaning(test_raw)
        
        # Aplicando o método de feature engineering
        df_2 = pipeline.feature_engineering(df_1)
        
        # Aplicando o método de feature engineering
        df_3 = pipeline.data_preparation(df_2)
        
        # Fazendo as predições
        df_response = pipeline.get_predictions(model, test_raw, df_3)
        
        
        return df_response # retorna o df para a pessoa que fez a requisição
        
        
    else: # se o dado da request não existir
        return Response('{}', status=200, mimetype='application/json') #retornando uma resposta


if __name__ == 'main': #iniciar a API
    app.run('0.0.0.0')
    #      (local host)  